In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import requests

In [2]:
base_url = 'https://api.binance.com' 

# saving the list of trading pairs

In [3]:
import pickle
#with open('trading_pairs.pkl', 'wb') as f:  # open a text file
#    pickle.dump(trading_pairs, f)
with open('trading_pairs.pkl', 'rb') as f:
    trading_pairs = pickle.load(f)

In [4]:
def mapinterval(interval):
    if interval == "1M":
        return "1Month"
    else:
        return interval

In [5]:
import requests
import csv
from datetime import datetime
from pathlib import Path
from os.path import exists
import time
base_url = 'https://api.binance.com' 
def get_klines(symbol, interval='1h', start_time=None, end_time=None, limit=500):
    endpoint = '/api/v3/klines'
    url = base_url + endpoint
    params = {'symbol': symbol,'interval': interval,'limit': limit}
    if start_time:
        params['startTime'] = start_time
    if end_time:
        params['endTime'] = end_time
    response = requests.get(url, params=params)
    if response.status_code == 200:
        klines_data = response.json()
        klines = []
        for kline in klines_data:
            klines.append({
                'open_time': kline[0],
                'open': float(kline[1]),
                'high': float(kline[2]),
                'low': float(kline[3]),
                'close': float(kline[4]),
                'volume': float(kline[5]),
                'close_time': kline[6],
                'quote_asset_volume': float(kline[7]),
                'number_of_trades': kline[8],
                'taker_buy_base_asset_volume': float(kline[9]),
                'taker_buy_quote_asset_volume': float(kline[10]),
                'ignore': kline[11]
            })

        return klines
    else:
        print(f'Error: {response.status_code}',end="")
        return None

def save_klines_to_csv(klines, filename,data_folder):
    Path(data_folder).mkdir(parents=True, exist_ok=True)
    fieldnames = [
        'open_time', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ]

    with open(data_folder+filename, mode='w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for kline in klines:
            writer.writerow(kline)
def doesFileExist(filename,data_folder):
    return exists(data_folder+filename)
def download_klines_yearly(symbol, interval='1h', start_year=2018,data_folder=None):
    current_year = datetime.now().year
    ms_per_year = 31536000 * 1000
    for year in range(start_year, current_year + 1):
        filename = f'{symbol}_{year}_{mapinterval(interval)}.csv'
        if doesFileExist(filename,data_folder) and current_year!=year:
            continue
        start_time = int(datetime(year, 1, 1).timestamp() * 1000)
        end_time = int(start_time + ms_per_year)
        klines = []
        last_fetched_time = start_time
        counts=0
        while last_fetched_time < end_time:
            fetched_klines = get_klines(symbol, interval, start_time=last_fetched_time, limit=1000)
            if counts%50==0 and counts//50>0:
                print(counts//50,end="")
            if counts%15==0 and counts//15>0:
                time.sleep(2)
            counts+=1
            if (not fetched_klines) or len(fetched_klines)==0:
                break
            #print(fetched_klines) 
            new_fetched_time = fetched_klines[-1]['open_time']
            if last_fetched_time==new_fetched_time:
                break
            last_fetched_time = new_fetched_time
            klines.extend(fetched_klines)
        if len(klines)>0:
            save_klines_to_csv(klines, filename ,data_folder)
            print("",end=" ")
        time.sleep(5)
    time.sleep(5)

## sample run

In [60]:
# symbol = 'BTCUSDT'
# data_folder=f"kline_data/{symbol}/"
# interval = '1h'
# download_klines_yearly(symbol, interval,data_folder=data_folder)

## large run, run for all intervals >= 2h

In [6]:
intervals=['1m','3m','5m','15m','30m','1h','2h','4h','6h','8h','12h','1d','3d','1w','1M']
#intervals=['12h','1d','3d','1w','1M']
#intervals=['5m','15m','30m','1h','2h','4h','6h','8h','12h','1d','3d','1w','1M']
#intervals=['5m']
#intervals=['1m','3m']
#intervals=['1m','3m','5m','15m','30m','1h','1M']#,'2h','4h','6h','8h','12h','1d']
#intervals=['6h','8h','12h','1d']
intervals=['5m']

In [ ]:
%%time
#intervals=['1m']#,'30m','1h','1M']
for interval in intervals[::-1]:
    print(interval)
    for i,sp in list(enumerate(trading_pairs))[:250]:
        symbol = sp["symbol"]
        data_folder=f"kline_data/{symbol}/"
        print(f",{i}{symbol}",end=" ")
        try:
            download_klines_yearly(symbol, interval,data_folder=data_folder,start_year=2023)
        except ConnectionResetError or ConnectionAbortedError:
            print("error, sleep 10mins")
            time.sleep(10*60)
            download_klines_yearly(symbol, interval,data_folder=data_folder)
    print("")

5m
,0BTCUSDT 1 ,1EDUUSDT  ,2ETHUSDT 1 ,3USDCUSDT  ,4BUSDUSDT 1 ,5XRPUSDT 1 ,6SOLUSDT 1 ,7BNBUSDT 1 ,8ARBUSDT  ,9ALPHAUSDT 1 ,10IDUSDT  ,11CFXUSDT 1 ,12TUSDUSDT  ,13RNDRUSDT 1 ,14APTUSDT 1 ,15INJUSDT 1 ,16DOGEUSDT 1 ,17MATICUSDT 1 ,18LTCUSDT 1 ,19RADUSDT 1 ,20ICPUSDT 1 ,21EURUSDT 1 ,22FILUSDT 1 ,23LINAUSDT 1 ,24LINKUSDT 1 ,25FTMUSDT 1 ,26DYDXUSDT 1 ,27ADAUSDT 1 ,28WOOUSDT 1 ,29TRXUSDT 1 ,30ATOMUSDT 1 ,31HOOKUSDT 1 ,32OGUSDT 1 ,33RDNTUSDT  ,34DOTUSDT 1 ,35GALAUSDT 1 ,36COCOSUSDT  ,37LUNAUSDT 1 ,38LUNCUSDT 1 ,39JASMYUSDT 1 ,40AVAXUSDT 1 ,41OPUSDT 1 ,42TOMOUSDT 1 ,43CAKEUSDT 1 ,44USTCUSDT  ,45EGLDUSDT 1 ,46IDEXUSDT 1 ,47FETUSDT 1 ,48SHIBUSDT 1 ,49GMTUSDT 1 ,50STMXUSDT 1 ,51AGIXUSDT  ,52ORNUSDT 1 ,53XMRUSDT 1 ,54BETAUSDT 1 ,55LQTYUSDT  ,56SANDUSDT 1 ,57STGUSDT 1 ,58VGXUSDT 1 ,59SXPUSDT 1 ,60BELUSDT 1 ,61MDTUSDT 1 ,62CTSIUSDT 1 ,63STXUSDT 1 ,64ROSEUSDT 1 ,65AUDUSDT  ,66ACHUSDT 1 ,67MAGICUSDT 1 ,68VTHOUSDT 1 ,69MASKUSDT 1 ,70EOSUSDT 1 ,71HIGHUSDT 1 ,72IMXUSDT 1 ,73NEARUSDT 1 ,74UMAUSDT 1 ,75G

# end of file

In [1]:
55000*5/60/24/30

6.36574074074074

In [120]:
%%time
for interval in intervals[::-1]:
    print(interval)
    for i,sp in list(enumerate(trading_pairs))[18:50]:
        symbol = sp["symbol"]
        data_folder=f"kline_data/{symbol}/"
        print(f",{i}{symbol}",end=" ")
        download_klines_yearly(symbol, interval,data_folder=data_folder)
    print("")

1M
,18LTCUSDT  ,19RADUSDT  ,20ICPUSDT  ,21EURUSDT  ,22FILUSDT  ,23LINAUSDT  ,24LINKUSDT  ,25FTMUSDT  ,26DYDXUSDT  ,27ADAUSDT  ,28WOOUSDT  ,29TRXUSDT  ,30ATOMUSDT  ,31HOOKUSDT  ,32OGUSDT  ,33RDNTUSDT  ,34DOTUSDT  ,35GALAUSDT  ,36COCOSUSDT  ,37LUNAUSDT  ,38LUNCUSDT  ,39JASMYUSDT  ,40AVAXUSDT  ,41OPUSDT  ,42TOMOUSDT  ,43CAKEUSDT  ,44USTCUSDT  ,45EGLDUSDT  ,46IDEXUSDT  ,47FETUSDT  ,48SHIBUSDT  ,49GMTUSDT  
1m
,18LTCUSDT 123 ,19RADUSDT 123 ,20ICPUSDT    123456 12345678910 123 ,21EURUSDT   12345678910 12345678910 12345678910 123 ,22FILUSDT   12 12345678910 12345678910 123 ,23LINAUSDT    12345678 12345678910 123 ,24LINKUSDT  12345678910 12345678910 12345678910 12345678910 123 ,25FTMUSDT  12345 12345678910 12345678910 12345678910 123 ,26DYDXUSDT    123 12345678910 123 ,27ADAUSDT 1234567 12345678910 12345678910 12345678910 12345678910 123 ,28WOOUSDT     123456789 123 ,29TRXUSDT 12345 12345678910 12345678910 12345678910 12345678910 123 ,30ATOMUSDT  1234567 12345678910 12345678910 12345678910 123

In [106]:
# import glob
# import os
# for file in glob.glob("kline_data/*/*1m*.csv"):
#     os.remove(file)

In [6]:
%%time
intervals=['5m']#,'30m','1h','1M']
for interval in intervals[::-1]:
    print(interval)
    for i,sp in list(enumerate(trading_pairs))[212:250]:
        symbol = sp["symbol"]
        data_folder=f"kline_data/{symbol}/"
        print(f",{i}{symbol}",end=" ")
        try:
            download_klines_yearly(symbol, interval,data_folder=data_folder)
        except ConnectionResetError or ConnectionAbortedError:
            print("error, sleep 10mins")
            time.sleep(10*60)
            download_klines_yearly(symbol, interval,data_folder=data_folder)
    print("")

5m
,212MCUSDT  ,213RENUSDT 12  ,214XEMUSDT    12 12  ,215KNCUSDT   1 12 12  ,216HIFIUSDT       ,217GLMRUSDT     12  ,218FLUXUSDT     12  ,219TLMUSDT    1 12  ,220KP3RUSDT     12  ,221MDXUSDT    1 12  ,222SUPERUSDT    1 12  ,223CTKUSDT    12 12  ,224PYRUSDT     12  ,225TUSDT     1  ,226ASRUSDT    12 12  ,227ILVUSDT     12  ,228ADXUSDT     12  ,229ZENUSDT   1 12 12  ,230BSWUSDT     1  ,231DFUSDT     12  ,232RPLUSDT       ,233JUVUSDT    12 12  ,234OGNUSDT   12 12 12  ,235YGGUSDT     12  ,236SFPUSDT    1 12  ,237SPELLUSDT     12  ,238IOSTUSDT  1 12 12 12  ,239PONDUSDT    1 12  ,240AMPUSDT     12  ,241BAKEUSDT    1 12  ,242YFIUSDT    12 12  ,243WAXPUSDT     12  ,244LPTUSDT    1 12  ,245TVKUSDT     12  ,246MOBUSDT     1  ,247BICOUSDT     12  ,248PLAUSDT     12  ,249SKLUSDT    12 12  
CPU times: total: 35min 9s
Wall time: 2h 13min 57s


In [7]:
%%time
intervals=['15m']#,'30m','1h','1M']
for interval in intervals[::-1]:
    print(interval)
    for i,sp in list(enumerate(trading_pairs))[232:250]:
        symbol = sp["symbol"]
        data_folder=f"kline_data/{symbol}/"
        print(f",{i}{symbol}",end=" ")
        try:
            download_klines_yearly(symbol, interval,data_folder=data_folder)
        except ConnectionResetError or ConnectionAbortedError:
            print("error, sleep 10mins")
            time.sleep(10*60)
            download_klines_yearly(symbol, interval,data_folder=data_folder)
    print("")

15m
,232RPLUSDT  ,233JUVUSDT       ,234OGNUSDT       ,235YGGUSDT       ,236SFPUSDT       ,237SPELLUSDT       ,238IOSTUSDT       ,239PONDUSDT       ,240AMPUSDT       ,241BAKEUSDT       ,242YFIUSDT       ,243WAXPUSDT       ,244LPTUSDT       ,245TVKUSDT       ,246MOBUSDT       ,247BICOUSDT       ,248PLAUSDT       ,249SKLUSDT       
CPU times: total: 5min 11s
Wall time: 19min 17s


In [9]:
%%time
intervals=['3m']#,'30m','1h','1M']
for interval in intervals[::-1]:
    print(interval)
    for i,sp in list(enumerate(trading_pairs))[187:250]:
        symbol = sp["symbol"]
        data_folder=f"kline_data/{symbol}/"
        print(f",{i}{symbol}",end=" ")
        try:
            download_klines_yearly(symbol, interval,data_folder=data_folder)
        except ConnectionResetError or ConnectionAbortedError:
            print("error, sleep 10mins")
            time.sleep(10*60)
            download_klines_yearly(symbol, interval,data_folder=data_folder)
    print("")

3m
,187QTUMUSDT 1 ,188RSRUSDT 1 ,189KMDUSDT 1 ,190LEVERUSDT     1 1 ,191KSMUSDT   1 123 123 1 ,192BLZUSDT   1 123 123 1 ,193LAZIOUSDT     123 1 ,194OSMOUSDT      1 ,195PORTOUSDT     123 1 ,196ERNUSDT    1 123 1 ,197CHESSUSDT     123 1 ,198CLVUSDT    1 123 1 ,199DUSKUSDT  1 123 123 123 1 ,200IOTXUSDT   123 123 123 1 ,201VOXELUSDT     123 1 ,202VIBUSDT       ,203BONDUSDT    1 123 1 ,204PHAUSDT    1 123 1 ,205VIDTUSDT    1 123 1 ,206AMBUSDT       ,207UTKUSDT    123 123 1 ,208AGLDUSDT     123 1 ,209BATUSDT  12 123 123 123 1 ,210WINUSDT  1 123 123 123 1 ,211DODOUSDT    123 123 1 ,212MCUSDT     123 1 ,213RENUSDT   123 123 123 1 ,214XEMUSDT    123 123 1 ,215KNCUSDT   1 123 123 1 ,216HIFIUSDT      1 ,217GLMRUSDT     123 1 ,218FLUXUSDT     123 1 ,219TLMUSDT    12 123 1 ,220KP3RUSDT     123 1 ,221MDXUSDT    12 123 1 ,222SUPERUSDT    12 123 1 ,223CTKUSDT    123 123 1 ,224PYRUSDT     123 1 ,225TUSDT     12 1 ,226ASRUSDT    123 123 1 ,227ILVUSDT     123 1 ,228ADXUSDT     123 1 ,229ZENUSDT   1 123 1

In [17]:
%%time
intervals=['1m']#,'30m','1h','1M']
for interval in intervals[::-1]:
    print(interval)
    for i,sp in list(enumerate(trading_pairs))[148:250]:
        symbol = sp["symbol"]
        data_folder=f"kline_data/{symbol}/"
        print(f",{i}{symbol}",end=" ")
        try:
            download_klines_yearly(symbol, interval,data_folder=data_folder)
        except ConnectionResetError or ConnectionAbortedError:
            print("error, sleep 10mins")
            time.sleep(10*60)
            download_klines_yearly(symbol, interval,data_folder=data_folder)
    print("")

1m
,148QIUSDT 123 ,149ENJUSDT 12345678910 123 ,150XTZUSDT  12 12345678910 12345678910 12345678910 123 ,151ATAUSDT    12345 12345678910 123 ,152FLOWUSDT    1234 12345678910 123 ,153WAVESUSDT  123456789 12345678910 12345678910 12345678910 123 ,154LRCUSDT   12345 12345678910 12345678910 123 ,155USDPUSDT    123 1234567 1 ,156FIDAUSDT    12 12345678910 123 ,157ARPAUSDT  1 12345678910 12345678910 12345678910 123 ,158SANTOSUSDT     12345678910 123 ,159HOTUSDT  123456789 12345678910 12345678910 12345678910 123 ,160ALICEUSDT    12345678 12345678910 123 ,161ZECUSDT  12345678 12345678910 12345678910 12345678910 123 ,162ALPINEUSDT     123456789 123 ,163COTIUSDT   12345678 12345678910 12345678910 123 ,164GTCUSDT    12345 12345678910 123 ,165IOTAUSDT 123456 12345678910 12345678910 12345678910 12345678910 123 ,166NULSUSDT 1234 12345678910 12345678910 12345678910 12345678910 123 ,167RVNUSDT  12 12345678910 12345678910 12345678910 123 ,168BURGERUSDT    1234567 12345678910 123 ,169LOKAUSDT     123456789